In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
torch.cuda.is_available()

True

In [2]:
# Check if CUDA is available
if torch.cuda.is_available():
    print("CUDA is available. Using GPU.")
    device = torch.device("cuda")
else:
    print("CUDA is not available. Using CPU.")
    device = torch.device("cpu")

CUDA is available. Using GPU.


In [3]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import time

In [4]:
transform = transforms.Compose([
    transforms.ToTensor(),
])

In [5]:
from torchvision import datasets
data_path = '../data-unversioned/p1ch7/'
cifar10 = datasets.CIFAR10(data_path, train=True, download=True, transform=transform)
cifar10_val = datasets.CIFAR10(data_path, train=False, download=True, transform=transform)


Files already downloaded and verified
Files already downloaded and verified


In [13]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.act1 = nn.Tanh()
        self.pool1 = nn.MaxPool2d(2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.act2 = nn.Tanh()
        self.pool2 = nn.MaxPool2d(2)
        self.fc1 = nn.Linear(64 * 8 * 8, 256)  
        self.act3 = nn.Tanh()
        self.fc2 = nn.Linear(256, 10)

    def forward(self, x):
        out = self.pool1(self.act1(self.conv1(x)))
        out = self.pool2(self.act2(self.conv2(out)))
        out = out.view(-1, 64 * 8 * 8) 
        out = self.act3(self.fc1(out))
        out = self.fc2(out)
        return out


In [46]:
import datetime

def validate(model, loader, device):
    correct = 0
    total = 0
    model.eval()  #
    with torch.no_grad(): 
        for imgs, labels in loader:
            imgs = imgs.to(device=device)
            labels = labels.to(device=device)
            outputs = model(imgs)
            _, predicted = torch.max(outputs, dim=1)
            total += labels.shape[0]
            correct += int((predicted == labels).sum())
    model.train()  
    return correct / total

def training_loop(n_epochs, optimizer, model, loss_fn, train_loader, val_loader, device):
    total_start_time = time.time()
    for epoch in range(1, n_epochs + 1):
        model.train()  
        loss_train = 0.0
        start_time = time.time()
        
        for imgs, labels in train_loader:
            imgs = imgs.to(device=device)    
            labels = labels.to(device=device) 
            outputs = model(imgs)
            loss = loss_fn(outputs, labels)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            loss_train += loss.item()
          
        val_accuracy = validate(model, val_loader, device) *100
        avg_loss = loss_train / len(train_loader)
        print(f"Epoch: {epoch}, Training Loss: {avg_loss}")
        print(f'Validation Accuracy after epoch {epoch}: {val_accuracy}%')
    end_time = time.time()  
    duration = end_time - total_start_time
    print(f'Total Training Time: {duration} seconds')

In [47]:
train_loader = DataLoader(cifar10, batch_size=64, shuffle=True)
val_loader = DataLoader(cifar10_val, batch_size=64, shuffle=False)

In [48]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=1e-2)
loss_fn = nn.CrossEntropyLoss()

In [49]:
training_loop(
    n_epochs=300,
    optimizer=optimizer,
    model=model,
    loss_fn=loss_fn,
    train_loader=train_loader,
    val_loader=val_loader,
    device=device
)

Epoch: 1, Training Loss: 2.065116420883657
Validation Accuracy after epoch 1: 33.83%
Epoch: 2, Training Loss: 1.7632340982442012
Validation Accuracy after epoch 2: 42.21%
Epoch: 3, Training Loss: 1.6107084688628117
Validation Accuracy after epoch 3: 42.08%
Epoch: 4, Training Loss: 1.5177295834512052
Validation Accuracy after epoch 4: 45.89%
Epoch: 5, Training Loss: 1.4411470807726732
Validation Accuracy after epoch 5: 44.89%
Epoch: 6, Training Loss: 1.3688853079705592
Validation Accuracy after epoch 6: 47.17%
Epoch: 7, Training Loss: 1.3083990704068138
Validation Accuracy after epoch 7: 54.24%
Epoch: 8, Training Loss: 1.2526253458788938
Validation Accuracy after epoch 8: 54.510000000000005%
Epoch: 9, Training Loss: 1.2037605140977503
Validation Accuracy after epoch 9: 55.82%
Epoch: 10, Training Loss: 1.1594743931384952
Validation Accuracy after epoch 10: 56.42%
Epoch: 11, Training Loss: 1.1188343331179655
Validation Accuracy after epoch 11: 59.31999999999999%
Epoch: 12, Training Loss: 

In [54]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.act1 = nn.Tanh()
        self.pool1 = nn.MaxPool2d(2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.act2 = nn.Tanh()
        self.pool2 = nn.MaxPool2d(2)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.act3 = nn.Tanh()
        self.pool3 = nn.MaxPool2d(2)
        self.fc1 = nn.Linear(128 * 4 * 4, 256)
        self.act4 = nn.Tanh()
        self.fc2 = nn.Linear(256, 10)

    def forward(self, x):
        out = self.pool1(self.act1(self.conv1(x)))
        out = self.pool2(self.act2(self.conv2(out)))
        out = self.pool3(self.act3(self.conv3(out)))
        out = out.view(-1, 128 * 4 * 4) 
        out = self.act4(self.fc1(out))
        out = self.fc2(out)
        return out


In [57]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=1e-2)
loss_fn = nn.CrossEntropyLoss()

In [58]:
training_loop(
    n_epochs=300,
    optimizer=optimizer,
    model=model,
    loss_fn=loss_fn,
    train_loader=train_loader,
    val_loader=val_loader,
    device=device
)

Epoch: 1, Training Loss: 2.1930723769585496
Validation Accuracy after epoch 1: 26.61%
Epoch: 2, Training Loss: 1.9090867981581432
Validation Accuracy after epoch 2: 37.01%
Epoch: 3, Training Loss: 1.7144578894995668
Validation Accuracy after epoch 3: 41.010000000000005%
Epoch: 4, Training Loss: 1.5864605941735874
Validation Accuracy after epoch 4: 44.86%
Epoch: 5, Training Loss: 1.4918663739548315
Validation Accuracy after epoch 5: 48.59%
Epoch: 6, Training Loss: 1.4155759745851502
Validation Accuracy after epoch 6: 51.06%
Epoch: 7, Training Loss: 1.3517482565796894
Validation Accuracy after epoch 7: 50.760000000000005%
Epoch: 8, Training Loss: 1.2963406500761465
Validation Accuracy after epoch 8: 54.42%
Epoch: 9, Training Loss: 1.2419960498047606
Validation Accuracy after epoch 9: 55.83%
Epoch: 10, Training Loss: 1.1934811812075203
Validation Accuracy after epoch 10: 54.14%
Epoch: 11, Training Loss: 1.1466546099051795
Validation Accuracy after epoch 11: 59.63%
Epoch: 12, Training Loss

In [79]:
#Question 2 a)
class ResBlock(nn.Module):
    def __init__(self, n_chans):
        super(ResBlock, self).__init__()
        self.conv = nn.Conv2d(n_chans, n_chans, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(n_chans, n_chans, kernel_size=3, padding=1)

    def forward(self, x):
        residual = x
        out = F.relu(self.conv(x))
        out = self.conv2(out)
        out += residual  
        return F.relu(out)

class ResNet10(nn.Module):
    def __init__(self, n_chans1=32, n_blocks=10):
        super(ResNet10, self).__init__()
        self.n_chans1 = n_chans1
        self.conv1 = nn.Conv2d(3, n_chans1, kernel_size=3, padding=1)
        self.resblocks = nn.Sequential(*[ResBlock(n_chans1) for _ in range(n_blocks)])
        
  
        self.fc1 = nn.Linear(n_chans1 * 8 * 8, 32)
        self.fc2 = nn.Linear(32, 10) 

    def forward(self, x):
        out = F.max_pool2d(torch.relu(self.conv1(x)), 2)  
        out = self.resblocks(out)  
        out = F.adaptive_avg_pool2d(out, (8, 8))  
        out = out.view(out.size(0), -1)  
        out = torch.relu(self.fc1(out))
        out = self.fc2(out)
        return out


In [80]:
model = ResNet10(n_chans1=32, n_blocks=10).to(device)
optimizer = optim.SGD(model.parameters(), lr=1e-2)
loss_fn = nn.CrossEntropyLoss()

In [81]:
training_loop(
    n_epochs=300,
    optimizer=optimizer,
    model=model,
    loss_fn=loss_fn,
    train_loader=train_loader,
    val_loader=val_loader,
    device=device
)

Epoch: 1, Training Loss: 2.0462962231977517
Validation Accuracy after epoch 1: 30.09%
Epoch: 2, Training Loss: 1.736056514865602
Validation Accuracy after epoch 2: 40.96%
Epoch: 3, Training Loss: 1.5505998003513306
Validation Accuracy after epoch 3: 38.690000000000005%
Epoch: 4, Training Loss: 1.4388548196734066
Validation Accuracy after epoch 4: 46.35%
Epoch: 5, Training Loss: 1.3462261626939944
Validation Accuracy after epoch 5: 47.949999999999996%
Epoch: 6, Training Loss: 1.268026067701447
Validation Accuracy after epoch 6: 53.63%
Epoch: 7, Training Loss: 1.2021813150257101
Validation Accuracy after epoch 7: 52.38%
Epoch: 8, Training Loss: 1.1465146859436084
Validation Accuracy after epoch 8: 58.379999999999995%
Epoch: 9, Training Loss: 1.0946092156650464
Validation Accuracy after epoch 9: 42.4%
Epoch: 10, Training Loss: 1.0533907853276527
Validation Accuracy after epoch 10: 49.99%
Epoch: 11, Training Loss: 1.0041152521624894
Validation Accuracy after epoch 11: 59.099999999999994%
E

In [82]:
#2b ) weight decay 
model = ResNet10(n_chans1=32, n_blocks=10).to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=1e-2, weight_decay=0.001)
loss_fn = nn.CrossEntropyLoss()

In [83]:
training_loop(
    n_epochs=300,
    optimizer=optimizer,
    model=model,
    loss_fn=loss_fn,
    train_loader=train_loader,
    val_loader=val_loader,
    device=device
)

Epoch: 1, Training Loss: 2.0863163876716437
Validation Accuracy after epoch 1: 31.540000000000003%
Epoch: 2, Training Loss: 1.7833486260355587
Validation Accuracy after epoch 2: 20.830000000000002%
Epoch: 3, Training Loss: 1.6215860675972746
Validation Accuracy after epoch 3: 43.1%
Epoch: 4, Training Loss: 1.5115983608128774
Validation Accuracy after epoch 4: 30.659999999999997%
Epoch: 5, Training Loss: 1.4346572540300278
Validation Accuracy after epoch 5: 49.519999999999996%
Epoch: 6, Training Loss: 1.3648563983952602
Validation Accuracy after epoch 6: 38.84%
Epoch: 7, Training Loss: 1.29775622372737
Validation Accuracy after epoch 7: 53.47%
Epoch: 8, Training Loss: 1.2440070105940484
Validation Accuracy after epoch 8: 48.72%
Epoch: 9, Training Loss: 1.1891172707385724
Validation Accuracy after epoch 9: 52.54%
Epoch: 10, Training Loss: 1.1430731984355567
Validation Accuracy after epoch 10: 54.169999999999995%
Epoch: 11, Training Loss: 1.0930197679478189
Validation Accuracy after epoch

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
torch.cuda.is_available()

True

In [3]:
#Question 2 b) drop out
class ResBlock(nn.Module):
    def __init__(self, n_chans):
        super(ResBlock, self).__init__()
        self.conv = nn.Conv2d(n_chans, n_chans, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(n_chans, n_chans, kernel_size=3, padding=1)
        self.dropout = nn.Dropout2d(p=0.3)
    def forward(self, x):
        residual = x
        out = F.relu(self.conv(x))
        out = self.dropout(out)
        out = self.conv2(out)
        out += residual  
        return F.relu(out)

class ResNet10(nn.Module):
    def __init__(self, n_chans1=32, n_blocks=10):
        super(ResNet10, self).__init__()
        self.n_chans1 = n_chans1
        self.conv1 = nn.Conv2d(3, n_chans1, kernel_size=3, padding=1)
        self.conv1_dropout = nn.Dropout2d(p=0.3)
        self.resblocks = nn.Sequential(*[ResBlock(n_chans1) for _ in range(n_blocks)])
        
  
        self.fc1 = nn.Linear(n_chans1 * 8 * 8, 32)
        self.fc2 = nn.Linear(32, 10) 

    def forward(self, x):
        out = F.max_pool2d(torch.relu(self.conv1(x)), 2) 
        out = self.conv1_dropout(out)
        out = self.resblocks(out)  
        out = F.adaptive_avg_pool2d(out, (8, 8))  
        out = out.view(out.size(0), -1)  
        out = torch.relu(self.fc1(out))
        out = self.fc2(out)
        return out


In [4]:
import datetime

def validate(model, loader, device):
    correct = 0
    total = 0
    model.eval()  #
    with torch.no_grad(): 
        for imgs, labels in loader:
            imgs = imgs.to(device=device)
            labels = labels.to(device=device)
            outputs = model(imgs)
            _, predicted = torch.max(outputs, dim=1)
            total += labels.shape[0]
            correct += int((predicted == labels).sum())
    model.train()  
    return correct / total

def training_loop(n_epochs, optimizer, model, loss_fn, train_loader, val_loader, device):
    total_start_time = time.time()
    for epoch in range(1, n_epochs + 1):
        model.train()  
        loss_train = 0.0
        start_time = time.time()
        
        for imgs, labels in train_loader:
            imgs = imgs.to(device=device)    
            labels = labels.to(device=device) 
            outputs = model(imgs)
            loss = loss_fn(outputs, labels)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            loss_train += loss.item()
          
        val_accuracy = validate(model, val_loader, device) *100
        avg_loss = loss_train / len(train_loader)
        print(f"Epoch: {epoch}, Training Loss: {avg_loss}")
        print(f'Validation Accuracy after epoch {epoch}: {val_accuracy}%')
    end_time = time.time()  
    duration = end_time - total_start_time
    print(f'Total Training Time: {duration} seconds')

In [6]:
# Check if CUDA is available
if torch.cuda.is_available():
    print("CUDA is available. Using GPU.")
    device = torch.device("cuda")
else:
    print("CUDA is not available. Using CPU.")
    device = torch.device("cpu")

CUDA is available. Using GPU.


In [23]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import time
import torch.nn as nn
import torch.nn.functional as F

In [24]:
from torchvision import datasets
data_path = '../data-unversioned/p1ch7/'
cifar10 = datasets.CIFAR10(data_path, train=True, download=True, transform=transform)
cifar10_val = datasets.CIFAR10(data_path, train=False, download=True, transform=transform)


Files already downloaded and verified
Files already downloaded and verified


In [25]:
transform = transforms.Compose([
    transforms.ToTensor(),
])

In [26]:
train_loader = DataLoader(cifar10, batch_size=64, shuffle=True)
val_loader = DataLoader(cifar10_val, batch_size=64, shuffle=False)

In [27]:
model = ResNet10(n_chans1=32, n_blocks=10).to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=1e-2)
loss_fn = nn.CrossEntropyLoss()

In [28]:
training_loop(
    n_epochs=300,
    optimizer=optimizer,
    model=model,
    loss_fn=loss_fn,
    train_loader=train_loader,
    val_loader=val_loader,
    device=device
)

Epoch: 1, Training Loss: 2.188420028332859
Validation Accuracy after epoch 1: 26.96%
Epoch: 2, Training Loss: 1.9077004058586666
Validation Accuracy after epoch 2: 37.91%
Epoch: 3, Training Loss: 1.7624874541826565
Validation Accuracy after epoch 3: 32.6%
Epoch: 4, Training Loss: 1.6527273520789183
Validation Accuracy after epoch 4: 38.73%
Epoch: 5, Training Loss: 1.5709573911583943
Validation Accuracy after epoch 5: 45.59%
Epoch: 6, Training Loss: 1.5084244925957506
Validation Accuracy after epoch 6: 41.839999999999996%
Epoch: 7, Training Loss: 1.4505255758914801
Validation Accuracy after epoch 7: 48.11%
Epoch: 8, Training Loss: 1.3951543010104344
Validation Accuracy after epoch 8: 51.62%
Epoch: 9, Training Loss: 1.3519776779062607
Validation Accuracy after epoch 9: 51.83%
Epoch: 10, Training Loss: 1.3127172520703367
Validation Accuracy after epoch 10: 53.49%
Epoch: 11, Training Loss: 1.2804870741903935
Validation Accuracy after epoch 11: 52.26%
Epoch: 12, Training Loss: 1.24477910385

In [50]:
#Question 2 b) Batch Normalization 
class ResBlock(nn.Module):
    def __init__(self, n_chans):
        super(ResBlock, self).__init__()
        self.conv = nn.Conv2d(n_chans, n_chans, kernel_size=3, padding=1, bias=False)  
        self.batch_norm = nn.BatchNorm2d(n_chans)
        
      
        nn.init.kaiming_normal_(self.conv.weight, nonlinearity='relu')
        nn.init.constant_(self.batch_norm.weight, 1)  
        nn.init.zeros_(self.batch_norm.bias)

    def forward(self, x):
        out = self.conv(x)
        out = self.batch_norm(out)
        out = F.relu(out)
        return out + x  

class NetResDeep(nn.Module):  
    def __init__(self, n_chans1=32, n_blocks=10):
        super(NetResDeep, self).__init__()
        self.n_chans1 = n_chans1
        self.conv1 = nn.Conv2d(3, n_chans1, kernel_size=3, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(n_chans1)
        
        
        nn.init.kaiming_normal_(self.conv1.weight, nonlinearity='relu')
        
     
        self.resblocks = nn.Sequential(*[ResBlock(n_chans1) for _ in range(n_blocks)])
        
       
        self.fc1 = nn.Linear(8 * 8 * n_chans1, 32)  
        self.fc2 = nn.Linear(32, 10)  

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))  
        out = F.max_pool2d(out, 2) 
        out = self.resblocks(out)  
        out = F.adaptive_avg_pool2d(out, (8, 8)) 
        out = out.view(-1, 8 * 8 * self.n_chans1) 
        out = F.relu(self.fc1(out))  
        out = self.fc2(out)  
        return out

In [51]:
model = NetResDeep(n_chans1=32, n_blocks=10).to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=3e-3)
loss_fn = nn.CrossEntropyLoss()

In [52]:
training_loop(
    n_epochs=300,
    optimizer=optimizer,
    model=model,
    loss_fn=loss_fn,
    train_loader=train_loader,
    val_loader=val_loader,
    device=device
)

Epoch: 1, Training Loss: 1.9562370304561332
Validation Accuracy after epoch 1: 30.45%
Epoch: 2, Training Loss: 1.6446315262018871
Validation Accuracy after epoch 2: 34.43%
Epoch: 3, Training Loss: 1.4922315705462794
Validation Accuracy after epoch 3: 41.6%
Epoch: 4, Training Loss: 1.400319983129916
Validation Accuracy after epoch 4: 44.39%
Epoch: 5, Training Loss: 1.335924232097538
Validation Accuracy after epoch 5: 52.239999999999995%
Epoch: 6, Training Loss: 1.2825282219120913
Validation Accuracy after epoch 6: 46.86%
Epoch: 7, Training Loss: 1.2375153729982693
Validation Accuracy after epoch 7: 50.67%
Epoch: 8, Training Loss: 1.1968341992639215
Validation Accuracy after epoch 8: 54.32%
Epoch: 9, Training Loss: 1.1643986580774301
Validation Accuracy after epoch 9: 45.550000000000004%
Epoch: 10, Training Loss: 1.1321535258341933
Validation Accuracy after epoch 10: 52.25%
Epoch: 11, Training Loss: 1.101977552065764
Validation Accuracy after epoch 11: 56.75%
Epoch: 12, Training Loss: 1.